In [1]:
import pandas as pd
import re
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from bs4 import BeautifulSoup
import re
import pandas as pd
#from tabulate import tabulate
import os
#import chromedriver_binary
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import datetime as dt
from dateutil.parser import parse

In [2]:
AS= pd.read_csv('../data/United_Airlines.csv',skiprows=7)


In [3]:
df= pd.DataFrame(AS)
print(len(df))

3574


In [4]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
303,UA,01/26/2015,233.0,NaN,LAX,00:38,00:00
304,UA,01/26/2015,275.0,NaN,LAX,16:29,00:00
305,UA,01/26/2015,502.0,NaN,SFO,01:14,00:00
306,UA,01/26/2015,704.0,NaN,SFO,05:26,00:00
307,UA,01/26/2015,716.0,NaN,SFO,22:42,00:00
309,UA,01/26/2015,760.0,NaN,SFO,19:27,00:00
310,UA,01/26/2015,779.0,NaN,LAX,18:53,00:00
311,UA,01/26/2015,824.0,NaN,SFO,06:56,00:00
312,UA,01/26/2015,867.0,NaN,LAX,21:28,00:00
314,UA,01/26/2015,912.0,NaN,LAX,06:02,00:00


In [5]:
df= df.dropna(subset=['Scheduled Arrival Time','Actual Arrival Time'])
df.head()

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,UA,01/01/2015,275.0,N598UA,LAX,16:30,16:06
1,UA,01/01/2015,304.0,N510UA,SFO,23:41,23:39
2,UA,01/01/2015,716.0,N589UA,SFO,21:55,21:57
3,UA,01/01/2015,760.0,N588UA,SFO,19:25,19:49
4,UA,01/01/2015,779.0,N595UA,LAX,18:50,18:52


In [6]:
#Rename it to scheduled arrival date
df.rename(columns = {'Date (MM/DD/YYYY)':'Scheduled Date'}, inplace = True) 


In [7]:
df.head()

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,UA,01/01/2015,275.0,N598UA,LAX,16:30,16:06
1,UA,01/01/2015,304.0,N510UA,SFO,23:41,23:39
2,UA,01/01/2015,716.0,N589UA,SFO,21:55,21:57
3,UA,01/01/2015,760.0,N588UA,SFO,19:25,19:49
4,UA,01/01/2015,779.0,N595UA,LAX,18:50,18:52


In [8]:
df['Scheduled Date']= pd.to_datetime(df['Scheduled Date'])
type(df['Scheduled Date'][0])
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,UA,2015-01-01,275.0,N598UA,LAX,16:30,16:06
1,UA,2015-01-01,304.0,N510UA,SFO,23:41,23:39
2,UA,2015-01-01,716.0,N589UA,SFO,21:55,21:57
3,UA,2015-01-01,760.0,N588UA,SFO,19:25,19:49
4,UA,2015-01-01,779.0,N595UA,LAX,18:50,18:52
...,...,...,...,...,...,...,...
3568,UA,2015-10-23,898.0,N598UA,SFO,17:25,17:02
3569,UA,2015-10-23,912.0,N587UA,LAX,06:30,06:27
3570,UA,2015-10-23,1285.0,N33103,LAX,21:29,20:58
3571,UA,2015-10-24,275.0,N568UA,LAX,16:30,16:06


In [9]:
# Makea new column called actual arrival date 
df['Actual Arrival Date'] = df['Scheduled Date']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
0,UA,2015-01-01,275.0,N598UA,LAX,16:30,16:06,2015-01-01
1,UA,2015-01-01,304.0,N510UA,SFO,23:41,23:39,2015-01-01
2,UA,2015-01-01,716.0,N589UA,SFO,21:55,21:57,2015-01-01
3,UA,2015-01-01,760.0,N588UA,SFO,19:25,19:49,2015-01-01
4,UA,2015-01-01,779.0,N595UA,LAX,18:50,18:52,2015-01-01
...,...,...,...,...,...,...,...,...
3568,UA,2015-10-23,898.0,N598UA,SFO,17:25,17:02,2015-10-23
3569,UA,2015-10-23,912.0,N587UA,LAX,06:30,06:27,2015-10-23
3570,UA,2015-10-23,1285.0,N33103,LAX,21:29,20:58,2015-10-23
3571,UA,2015-10-24,275.0,N568UA,LAX,16:30,16:06,2015-10-24


In [10]:
df.loc[df['Actual Arrival Time']=='24:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date


In [11]:
df['Actual Arrival Time']= df['Actual Arrival Time'].str.replace('24:00','00:00')

In [12]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
303,UA,2015-01-26,233.0,NaN,LAX,00:38,00:00,2015-01-26
304,UA,2015-01-26,275.0,NaN,LAX,16:29,00:00,2015-01-26
305,UA,2015-01-26,502.0,NaN,SFO,01:14,00:00,2015-01-26
306,UA,2015-01-26,704.0,NaN,SFO,05:26,00:00,2015-01-26
307,UA,2015-01-26,716.0,NaN,SFO,22:42,00:00,2015-01-26
309,UA,2015-01-26,760.0,NaN,SFO,19:27,00:00,2015-01-26
310,UA,2015-01-26,779.0,NaN,LAX,18:53,00:00,2015-01-26
311,UA,2015-01-26,824.0,NaN,SFO,06:56,00:00,2015-01-26
312,UA,2015-01-26,867.0,NaN,LAX,21:28,00:00,2015-01-26
314,UA,2015-01-26,912.0,NaN,LAX,06:02,00:00,2015-01-26


In [13]:
df.loc[df['Actual Arrival Time']=='00:00','Actual Arrival Date']+=dt.timedelta(days=1)

In [14]:
df['Actual Arrival Date']= df['Actual Arrival Date'].dt.strftime('%Y-%m-%d')

In [15]:
df['Scheduled Date']= df['Scheduled Date'].dt.strftime('%Y-%m-%d')

In [16]:
df['Actual Arrival']=df['Actual Arrival Date'] + ' ' + df['Actual Arrival Time']

In [17]:
df['Scheduled Arrival']=df['Scheduled Date'] + ' ' + df['Scheduled Arrival Time']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,UA,2015-01-01,275.0,N598UA,LAX,16:30,16:06,2015-01-01,2015-01-01 16:06,2015-01-01 16:30
1,UA,2015-01-01,304.0,N510UA,SFO,23:41,23:39,2015-01-01,2015-01-01 23:39,2015-01-01 23:41
2,UA,2015-01-01,716.0,N589UA,SFO,21:55,21:57,2015-01-01,2015-01-01 21:57,2015-01-01 21:55
3,UA,2015-01-01,760.0,N588UA,SFO,19:25,19:49,2015-01-01,2015-01-01 19:49,2015-01-01 19:25
4,UA,2015-01-01,779.0,N595UA,LAX,18:50,18:52,2015-01-01,2015-01-01 18:52,2015-01-01 18:50
...,...,...,...,...,...,...,...,...,...,...
3568,UA,2015-10-23,898.0,N598UA,SFO,17:25,17:02,2015-10-23,2015-10-23 17:02,2015-10-23 17:25
3569,UA,2015-10-23,912.0,N587UA,LAX,06:30,06:27,2015-10-23,2015-10-23 06:27,2015-10-23 06:30
3570,UA,2015-10-23,1285.0,N33103,LAX,21:29,20:58,2015-10-23,2015-10-23 20:58,2015-10-23 21:29
3571,UA,2015-10-24,275.0,N568UA,LAX,16:30,16:06,2015-10-24,2015-10-24 16:06,2015-10-24 16:30


In [18]:
FMT = '%Y-%m-%d %H:%M'
df['Scheduled Arrival']= pd.to_datetime(df['Scheduled Arrival'], format=FMT)
df['Actual Arrival']= pd.to_datetime(df['Actual Arrival'], format=FMT)
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,UA,2015-01-01,275.0,N598UA,LAX,16:30,16:06,2015-01-01,2015-01-01 16:06:00,2015-01-01 16:30:00
1,UA,2015-01-01,304.0,N510UA,SFO,23:41,23:39,2015-01-01,2015-01-01 23:39:00,2015-01-01 23:41:00
2,UA,2015-01-01,716.0,N589UA,SFO,21:55,21:57,2015-01-01,2015-01-01 21:57:00,2015-01-01 21:55:00
3,UA,2015-01-01,760.0,N588UA,SFO,19:25,19:49,2015-01-01,2015-01-01 19:49:00,2015-01-01 19:25:00
4,UA,2015-01-01,779.0,N595UA,LAX,18:50,18:52,2015-01-01,2015-01-01 18:52:00,2015-01-01 18:50:00
...,...,...,...,...,...,...,...,...,...,...
3568,UA,2015-10-23,898.0,N598UA,SFO,17:25,17:02,2015-10-23,2015-10-23 17:02:00,2015-10-23 17:25:00
3569,UA,2015-10-23,912.0,N587UA,LAX,06:30,06:27,2015-10-23,2015-10-23 06:27:00,2015-10-23 06:30:00
3570,UA,2015-10-23,1285.0,N33103,LAX,21:29,20:58,2015-10-23,2015-10-23 20:58:00,2015-10-23 21:29:00
3571,UA,2015-10-24,275.0,N568UA,LAX,16:30,16:06,2015-10-24,2015-10-24 16:06:00,2015-10-24 16:30:00


In [19]:
df['Difference']= (df['Scheduled Arrival']-df['Actual Arrival']).astype('timedelta64[m]')
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival,Difference
0,UA,2015-01-01,275.0,N598UA,LAX,16:30,16:06,2015-01-01,2015-01-01 16:06:00,2015-01-01 16:30:00,24.0
1,UA,2015-01-01,304.0,N510UA,SFO,23:41,23:39,2015-01-01,2015-01-01 23:39:00,2015-01-01 23:41:00,2.0
2,UA,2015-01-01,716.0,N589UA,SFO,21:55,21:57,2015-01-01,2015-01-01 21:57:00,2015-01-01 21:55:00,-2.0
3,UA,2015-01-01,760.0,N588UA,SFO,19:25,19:49,2015-01-01,2015-01-01 19:49:00,2015-01-01 19:25:00,-24.0
4,UA,2015-01-01,779.0,N595UA,LAX,18:50,18:52,2015-01-01,2015-01-01 18:52:00,2015-01-01 18:50:00,-2.0
...,...,...,...,...,...,...,...,...,...,...,...
3568,UA,2015-10-23,898.0,N598UA,SFO,17:25,17:02,2015-10-23,2015-10-23 17:02:00,2015-10-23 17:25:00,23.0
3569,UA,2015-10-23,912.0,N587UA,LAX,06:30,06:27,2015-10-23,2015-10-23 06:27:00,2015-10-23 06:30:00,3.0
3570,UA,2015-10-23,1285.0,N33103,LAX,21:29,20:58,2015-10-23,2015-10-23 20:58:00,2015-10-23 21:29:00,31.0
3571,UA,2015-10-24,275.0,N568UA,LAX,16:30,16:06,2015-10-24,2015-10-24 16:06:00,2015-10-24 16:30:00,24.0


In [20]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2019
Count_2019=df[df['Actual Arrival Date'].str.contains("2019")]
df19= pd.DataFrame(Count_2019)
Total_19=len(df19)
Twenty_Mins_Early_19= len(df19[df19['Difference'] > 20.0])
Early_Arrivals_19= len(df19[df19['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2019:",Total_19)
print("Flights that arrived early in 2019:",Early_Arrivals_19)
print("Flights that arrived more than 20 mins early in 2019:",Twenty_Mins_Early_19)

Total Alaska Airlines flights that landed at JFK in 2019: 0
Flights that arrived early in 2019: 0
Flights that arrived more than 20 mins early in 2019: 0


In [21]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2018
Count_2018=df[df['Actual Arrival Date'].str.contains("2018")]
df18= pd.DataFrame(Count_2018)
Total_18=len(df18)
Twenty_Mins_Early_18= len(df18[df18['Difference'] > 20.0])
Early_Arrivals_18= len(df18[df18['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2018:",Total_18)
print("Flights that arrived early in 2018:",Early_Arrivals_18)
print("Flights that arrived more than 20 mins early in 2018:",Twenty_Mins_Early_18)

Total Alaska Airlines flights that landed at JFK in 2018: 0
Flights that arrived early in 2018: 0
Flights that arrived more than 20 mins early in 2018: 0


In [22]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2017
Count_2017=df[df['Actual Arrival Date'].str.contains("2017")]
df17= pd.DataFrame(Count_2017)
Total_17=len(df17)
Twenty_Mins_Early_17= len(df17[df17['Difference'] > 20.0])
Early_Arrivals_17= len(df17[df17['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2017:",Total_17)
print("Flights that arrived early in 2017:",Early_Arrivals_17)
print("Flights that arrived more than 20 mins early in 2017:",Twenty_Mins_Early_17)

Total Alaska Airlines flights that landed at JFK in 2017: 0
Flights that arrived early in 2017: 0
Flights that arrived more than 20 mins early in 2017: 0


In [23]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2016
Count_2016=df[df['Actual Arrival Date'].str.contains("2016")]
df16= pd.DataFrame(Count_2016)
Total_16=len(df16)
Twenty_Mins_Early_16= len(df16[df16['Difference'] > 20.0])
Early_Arrivals_16= len(df16[df16['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2016:",Total_16)
print("Flights that arrived early in 2016:",Early_Arrivals_16)
print("Flights that arrived more than 20 mins early in 2016:",Twenty_Mins_Early_16)

Total Alaska Airlines flights that landed at JFK in 2016: 0
Flights that arrived early in 2016: 0
Flights that arrived more than 20 mins early in 2016: 0


In [24]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2015
Count_2015=df[df['Actual Arrival Date'].str.contains("2015")]
df15= pd.DataFrame(Count_2015)
Total_15=len(df15)
Twenty_Mins_Early_15= len(df15[df15['Difference'] > 20.0])
Early_Arrivals_15= len(df15[df15['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2015:",Total_15)
print("Flights that arrived early in 2015:",Early_Arrivals_15)
print("Flights that arrived more than 20 mins early in 2015:",Twenty_Mins_Early_15)

Total Alaska Airlines flights that landed at JFK in 2015: 3573
Flights that arrived early in 2015: 2483
Flights that arrived more than 20 mins early in 2015: 960


In [25]:


# dictionary of lists  
 
year =["2015","2016","2017","2018","2019"]
total_flights= [Total_15, Total_16, Total_17, Total_18, Total_19]
early_arrival=[Early_Arrivals_15,Early_Arrivals_16,Early_Arrivals_17,Early_Arrivals_18,Early_Arrivals_19]
inflated_padding=[Twenty_Mins_Early_15,Twenty_Mins_Early_16,Twenty_Mins_Early_17,Twenty_Mins_Early_18,Twenty_Mins_Early_19]

dict = {'year': year, 'Total flights': total_flights, 'Early Arrivals': early_arrival, 'Inflated padding(>20mins early)': inflated_padding} 
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv(r'../output/United_Airlines.csv',index=False) 